#### Abfrage und Erstellen des Reddit-Features - von Simon Faltenberg



In [20]:
pip install pmaw

Note: you may need to restart the kernel to use updated packages.


Die API-Abfrage, um Kommentare in das DataFrame zu schreiben

In [ ]:
import pandas as pd
from pmaw import PushshiftAPI

 Da die Reddit-API nicht so funktioniert hat wie erwartet, wurde die PushshiftAPI genutzt. Diese API hat keine max. Grenze an Request womit wir dann um die 930.000 Kommentare abfragen konnten. Aus repräsentativen Zwecke, wurde ein Limit auf 2000 Kommentaren gesetzt um den Code trotzdem veranschaulichen zu können.


Erstellen der Variablen für die Zeitraumspanne, subreddit und query
API Abfrage. Anschließend wird das  Ergebnis in ein DataFrame geschrieben:

In [41]:
api = PushshiftAPI()


import datetime as dt
before = int(dt.datetime(2021,1,11,0,0).timestamp()) #last date to get comments from
after = int(dt.datetime(2021,1,7,0,0).timestamp()) #first date to get comments from



subreddit="wallstreetbets" #reddit group 
query = ["Gamestop | GME"] #words to search in comments
limit=2000 
comments = api.search_comments(q = query, subreddit=subreddit, limit=limit, before=before, after=after) #api request to get comments
print(f'Retrieved {len(comments)} comments from Pushshift') #feedback from terminal

comments_df = pd.DataFrame(comments) #create the DataFrame

Total:: Success Rate: 100.00% - Requests: 23 - Batches: 3 - Items Remaining: 0
Retrieved 2000 comments from Pushshift


Aufbau des DataFrame

In [42]:
comments_df.head(3)

,all_awardings,associated_award,author,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_template_id,author_flair_text,author_flair_text_color,author_flair_type,...,score,send_replies,stickied,subreddit,subreddit_id,top_awarded_type,total_awards_received,treatment_tags,distinguished,author_cakeday
0,[],None,All_I_Eat_Is_Gucci,None,None,[],None,None,None,text,...,1,True,False,wallstreetbets,t5_2th52,None,0,[],NaN,NaN
1,[],None,TheLilThnake,None,None,[],None,None,None,text,...,6,True,False,wallstreetbets,t5_2th52,None,0,[],NaN,NaN
2,[],None,RoyalFail6,None,None,[],None,None,None,text,...,2,True,False,wallstreetbets,t5_2th52,None,0,[],NaN,NaN


Der Counter um die Anzahl Kommentare pro Tag zu errechnen

In [32]:
import datetime

Neues DataFrame mit den drei entscheidenden Attributen erstellen

In [43]:
df = pd.DataFrame(comments_df)
df = df[['author', 'body', 'created_utc']] #overwrite df and keep just named variables
df.head(5)

,author,body,created_utc
0,All_I_Eat_Is_Gucci,"Hope my GME calls recover a bit, and hope ICLN...",1610008485
1,TheLilThnake,i’m calling it. GME to $20 tmmr before close.,1610008422
2,RoyalFail6,"Hopefully it doesn’t fall like Purple, GME, PL...",1610008304
3,Funguyguy,"Burry owns 5% GME, so 50 vix 50 GME?",1610007861
4,Nasty_Nate2324,How is it bullshit if there was no opinion mad...,1610007412


Umrechnung der UNIX-Zeit zum normalen Datum.

In [44]:
df["timestamp"] = pd.to_datetime(df["created_utc"], unit="s") #get the timestamp
df["just_date"] = df["timestamp"].dt.date 
df.head(5)

,author,body,created_utc,timestamp,just_date
0,All_I_Eat_Is_Gucci,"Hope my GME calls recover a bit, and hope ICLN...",1610008485,2021-01-07 08:34:45,2021-01-07
1,TheLilThnake,i’m calling it. GME to $20 tmmr before close.,1610008422,2021-01-07 08:33:42,2021-01-07
2,RoyalFail6,"Hopefully it doesn’t fall like Purple, GME, PL...",1610008304,2021-01-07 08:31:44,2021-01-07
3,Funguyguy,"Burry owns 5% GME, so 50 vix 50 GME?",1610007861,2021-01-07 08:24:21,2021-01-07
4,Nasty_Nate2324,How is it bullshit if there was no opinion mad...,1610007412,2021-01-07 08:16:52,2021-01-07


Zählen der erstellten Kommentare pro Tag

In [45]:
counter = df.groupby("just_date").size().rename("Count") #count comments per day

result = df.drop_duplicates(subset='just_date')\
    .merge(counter, left_on='just_date', right_index=True) #keep one comment for every day
result = result[['just_date', 'Count']] #keep the variable 'just date' and 'Count'
result

,just_date,Count
0,2021-01-07,640
200,2021-01-08,612
500,2021-01-09,332
693,2021-01-10,415
1092,2021-01-06,1
